In [8]:
import os

In [9]:
%pwd

'/mnt/c/projet/Kideny_Disease_Classification'

In [10]:
os.chdir("../")

In [11]:
%pwd

'/mnt/c/projet'

In [12]:
import dagshub
dagshub.init(repo_owner='RymTangour', repo_name='Kideny_Disease_Classification', mlflow=True)


Initialized MLflow to track repo "RymTangour/Kideny_Disease_Classification"

Repository RymTangour/Kideny_Disease_Classification initialized!

In [13]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model("artifacts/training/model.keras")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'artifacts/training/model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml ,create_directories, save_json

In [ ]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/demo kideny datset",
            mlflow_uri="https://dagshub.com/RymTangour/Kideny_Disease_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

        )
        return eval_config


In [ ]:
import tensorflow as tf 
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from mlflow import MlflowClient



In [ ]:
import tensorflow as tf 
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
print("GPU available:", tf.config.list_physical_devices('GPU'))

tf.Tensor(-884.0077, shape=(), dtype=float32)
GPU available: []


In [ ]:
print(tf.config.list_physical_devices('GPU'))

[]


In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.client=MlflowClient()

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

    def assign_alias_to_stage(self, model_name: str, stage: str, alias: str):
        """
        Assign an alias to the latest model version in a specific stage.

        Args:
            model_name (str): Name of the registered model.
            stage (str): Stage of the model (e.g., "Staging", "Production").
            alias (str): Alias to assign (e.g., "champion").
        """
        latest_mv = self.client.get_latest_versions(model_name, stages=[stage])[0]
        self.client.set_registered_model_alias(model_name, alias, latest_mv.version)

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    evaluation.assign_alias_to_stage("VGG16Model", "Staging", "champion")
except Exception as e:
   
    raise e

[2025-02-23 18:50:30,456:INFO:common:yaml file :config\config.yaml loaded succfully]
[2025-02-23 18:50:30,467:INFO:common:yaml file :params.yaml loaded succfully]


[2025-02-23 18:50:30,471:INFO:common:created directory at: artifacts]
Found 84 images belonging to 2 classes.
6/6 [==============================] - 10s 1s/step - loss: 3.5479e-08 - accuracy: 1.0000
[2025-02-23 18:50:40,831:INFO:common:json file saved at:scores.json]


2025/02/23 18:50:41 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\RYM\AppData\Local\Temp\tmp83wzmx7x\model\data\model\assets
[2025-02-23 18:50:43,377:INFO:builder_impl:Assets written to: C:\Users\RYM\AppData\Local\Temp\tmp83wzmx7x\model\data\model\assets]


2025/02/23 18:50:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\RYM\AppData\Local\Temp\tmp83wzmx7x\model, flavor: tensorflow). Fall back to return ['tensorflow==2.18.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2025/02/23 18:50:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-02-23 18:51:52,744:WARNING:connectionpool:Retrying (Retry(total=6, connect=7, read=7, redirect=7, status=7)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2426)')': /RymTangour/Kideny_Disease_Classification.mlflow/api/2.0/mlflow-artifacts/artifacts/a86bfbcd9098411f95a8c8439f64c854/09a00ebbf1dd490d9564da34f4fc72a2/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2025-02-23 18:52:27,933:WARNING:connectionpool:Retrying (Retry(total=5, connect=7, read=7, redirect=7, status=7)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2426)')': /RymTangour/Kideny_Disease_Classification.mlflow/api/2.0/mlflow-artifacts/artifacts/a86bfbcd9098411f95a8c8439f64c854/09a00ebbf1dd490d9564da34f4fc72a2/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2025-02-23 18:53:13,927:WARNING:connectionpool:Retrying (Retry(total=4, connect=7, read=7, redirect=7, status=7)) after connection br

In [ ]:
import tensorflow as tf  

# Check for GPU availability
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

# Get GPU details
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print("GPU Name:", tf.config.experimental.get_device_details(gpus[0])['device_name'])

# Select device
device = "/GPU:0" if gpus else "/CPU:0"
print("Using device:", device)

In [15]:
"""import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)"""

'import torch\n\nprint("Number of GPU: ", torch.cuda.device_count())\nprint("GPU Name: ", torch.cuda.get_device_name())\n\n\ndevice = torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\')\nprint(\'Using device:\', device)'

In [ ]:
import tensorflow as tf  

# Check for GPU availability
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

# Get GPU details
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print("GPU Name:", tf.config.experimental.get_device_details(gpus[0])['device_name'])

# Select device
device = "/GPU:0" if gpus else "/CPU:0"
print("Using device:", device)

Num GPUs Available: 1
GPU Name: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Using device: /GPU:0
